# Consolidate data (Batch1)

<div style="text-align: right"> Author: Vassil Dimitrov </div>
<div style="text-align: right"> Date: 2023-08-01 </div>

Batch 1 of the data will be prepared by merging the patients_batch1 data with the corresponding patient data from all other available tables. Note that some preprocessing will be required for some of the tables encluding `medications`, `observations` and `procedures`.

## Prep

### Load libraries

In [1]:
import numpy as np
import pandas as pd

---

### Load patients data

In [ ]:
b = pd.read_pickle('patients_batch1.pkl')

In [ ]:
b.shape

In [ ]:
display(b.head())

Obtain column names from the patients dataframe for downstream analyses.

In [ ]:
patients_dat_cols = list(b.columns)
print(patients_dat_cols)

---

## Functions

### `PATIENT` to index & explode

In [19]:
def patient_subset_index_explode(df1, df2):
    # Subset data
    print('subsetting...')
    tab = df2[df2['PATIENT'].isin(df1.index)]
    print('column types:', tab.dtypes)
    del df2
    print('Subset table dimensions:', tab.shape)
    
    # Convert data to dense:
    for col in list(tab.columns):        
        if pd.api.types.is_sparse(tab[col]):
            print(f'Processing column {col}...')
            tab[col] = tab[col].sparse.to_dense().astype(np.uint32)
        else:
            continue
        
    
    # Create a new column 'encounter' to differentiate between duplicate rows for each patient
    print('Counting number of encounters per patient...')
    tab['encounter_n'] = tab.groupby('PATIENT').cumcount()+1
    print('Pivoting table and setting "PATIENT" as index')
    tab = tab.pivot_table(index='PATIENT',
                          columns='encounter_n',
                          aggfunc='first')
    print(tab.shape)
    # Flatten the MultiIndex columns
    tab.columns = [f'{col[0]}_{col[1]}' for col in tab.columns]
    print('column types:', tab.dtypes)
    # Fill in nulls with 0
    print('Filling in nulls with 0s')
    tab.fillna(0, inplace=True)
    # Convert to uint32
    tab = tab.astype('uint32')
    # Return processed and subset table
    return tab

### Batch merge

In [3]:
def batch_merge (df1, tab):
    # Merge the data with consolidated dataframe
    print('Merging...')
    df1 = df1.merge(tab, 
                  left_index=True, 
                  right_index=True, 
                  how='left')
    print('Merged!')
    print('New dimensions:', df1.shape)
    return df1

### Load data

In [4]:
def load_table (tab_name):
    to_merge = pd.read_pickle(tab_name)
    print(f'Dimensions for {tab_name}:')
    print(to_merge.shape)
    return to_merge

### Subset batch

In [5]:
def subset_batch_merge (df1, df2):
    print('Merging...')
    df_b = df2[df2.index.isin(df1.index)]
    df1 = df1.merge(df_b, 
                  left_index=True, 
                  right_index=True, 
                  how='left')
    print('Merged!')
    return df1

### Tidy up

In [6]:
def tidy_up (df, cols2exclude):
    print('Filling up with 0s')
    df = df.fillna(0)
    cols = [col for col in list(df.columns) if col not in cols2exclude]
    for col in cols:
        print(f'Processing column {col}...')
        if pd.api.types.is_sparse(df[col]):
            df[col] = df[col].sparse.to_dense().astype(np.uint32)
        else:
            df[col] = df[col].astype(np.uint32)
        print(f'Processed column {col}.')
    return df

### Process and merge
Uses the three other functions defined above: `load_table`, `subset_batch_merge` and `tidy_up`.

In [7]:
def process_n_merge_processed (df_a, tab_name_b):
    cols2exclude=list(df_a.columns)
    df_b = load_table(tab_name_b)
    df_a = subset_batch_merge(df_a, df_b)
    df_a = tidy_up(df_a, cols2exclude)
    return df_a

---
---

---
---

## Allergies

`allergies` is already preprocessed so it will essentially be loaded, the corresponding patients will be identified based on the `patients_batch1` information, and their respecive data will be merged to the patients data based on the indexes of both tables, keeping all batch1 patients.

### Read table

In [ ]:
to_merge = pd.read_pickle('allergies_encounters.pkl')
# Sanity check
print('dimensions:', to_merge.shape)
display(to_merge.head())

### Subset for batch1 patients only

In [ ]:
to_merge_b = to_merge[to_merge.index.isin(b.index)]
# Sanity check
print('dimensions:', to_merge_b.shape)
display(to_merge_b.head())

A bit over 12,000 patients from batch1 also have information about allergies.

### Merge `allergies` to batch1

In [ ]:
batch = b.merge(to_merge_b, 
                  left_index=True, 
                  right_index=True, 
                  how='left')
# Sanity check:
print(batch.shape)
display(batch.head())

In [ ]:
# Clean up memory
del b, to_merge, to_merge_b

### Tidy up table

Replace nulls with 0 (no allergy encounter with a physician).

In [ ]:
batch.fillna(0, inplace=True)
display(batch.head())

Check the variable types for each column.

In [ ]:
batch.dtypes

The allergy types were transformed into `float64` type. They will be converted to `uint32` to save memory.

In [ ]:
for col in batch.columns:
    if col not in patients_dat_cols:
        print('Converting', col)
        batch[col] = batch[col].astype('uint32')
    else:
        continue

In [ ]:
batch.dtypes

The types for the merged dataframe are of appropriate value.  
  
  The column names will be saved to exclude for downstream data manipulation.

In [ ]:
cols2exclude = list(batch.columns)
print(cols2exclude)

---
---

## Careplans

In [ ]:
careplans = load_table('careplans_encounters.pkl')

In [ ]:
careplans.dtypes

In [ ]:
print(batch.shape)
cols2exclude = list(batch.columns)
batch = subset_batch_merge(batch, careplans)
print(batch.shape)

In [ ]:
batch.dtypes

In [ ]:
batch = tidy_up(batch, cols2exclude)
batch.dtypes

In [ ]:
print('Dimensions so far:', batch.shape)

In [ ]:
# Clean up memory
del careplans, cols2exclude

---
---

## Conditions

In [ ]:
batch = process_n_merge_processed (batch, 'conditions_encounters.pkl')
print('Dimensions so far:', batch.shape)

---
---

## Immunizations

In [ ]:
batch = process_n_merge_processed (batch, 'immunizations_encounters.pkl')
print('Dimensions so far:', batch.shape)

---
---

## Save

Delete the batch table so far to liberate memory for processing the next table - `observations`.

In [ ]:
batch.to_pickle('batch1.pkl')
del batch

## Observations

In [8]:
observations = load_table('observations_encounters.pkl')

Dimensions for observations_encounters.pkl:
(62209964, 64)


In [9]:
display(observations.head())

,Abuse Status [OMAHA],Alzheimer's disease (disorder),American house dust mite IgE Ab in Serum,Are you covered by health insurance or some other kind of health care plan [PhenX],Body Height,Body Mass Index,Body Weight,Calcium,Carbon Dioxide,Cardiac Arrest,...,Stroke,Systolic Blood Pressure,Total Cholesterol,Total score [MMSE],Triglycerides,Urea Nitrogen,Walnut IgE Ab in Serum,Wheat IgE Ab in Serum,White oak IgE Ab in Serum,PATIENT
0,0,0,0,0,2477883,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33f33990-ae8b-4be8-938f-e47ad473abfe
1,0,0,0,0,1822198,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,36d131ee-dd5b-4acb-acbe-19961c32c099
2,0,0,0,0,0,0,2477883,0,0,0,...,0,0,0,0,0,0,0,0,0,33f33990-ae8b-4be8-938f-e47ad473abfe
3,0,0,0,0,0,0,1822198,0,0,0,...,0,0,0,0,0,0,0,0,0,36d131ee-dd5b-4acb-acbe-19961c32c099
4,0,0,0,0,0,2477883,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33f33990-ae8b-4be8-938f-e47ad473abfe


In [10]:
observations.dtypes

Abuse Status [OMAHA]                                                                  uint32
Alzheimer's disease (disorder)                                                        uint32
American house dust mite IgE Ab in Serum                                              uint32
Are you covered by health insurance or some other kind of health care plan [PhenX]    uint32
Body Height                                                                           uint32
                                                                                       ...  
Urea Nitrogen                                                                         uint32
Walnut IgE Ab in Serum                                                                uint32
Wheat IgE Ab in Serum                                                                 uint32
White oak IgE Ab in Serum                                                             uint32
PATIENT                                                               

In [11]:
batch = pd.read_pickle('batch1.pkl')
display(batch.head())

,MARITAL,GENDER,m_since_birth,RACE_asian,RACE_black,RACE_hispanic,RACE_native,RACE_white,ETHNICITY_african,ETHNICITY_american,...,White oak IgE Ab in Serum_232,White oak IgE Ab in Serum_233,White oak IgE Ab in Serum_234,White oak IgE Ab in Serum_235,White oak IgE Ab in Serum_236,White oak IgE Ab in Serum_237,White oak IgE Ab in Serum_238,White oak IgE Ab in Serum_239,White oak IgE Ab in Serum_240,White oak IgE Ab in Serum_241
PATIENT,,,,,,,,,,,,,,,,,,,,,
eec94708-21e3-4303-871e-7e4963790448,1,1,398,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20fc6de1-def9-41b5-b415-42507e641334,0,0,666,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1e0b4c6c-9f9c-4b74-9efe-4d32bb6f27c4,0,0,87,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8c8b71ed-de7e-46e7-a5c2-705d7800477b,0,1,699,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
fab76a11-1220-4236-82a6-a7af36d19c4f,0,1,74,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
cols2exclude = list(batch.columns)

In [ ]:
df2 = patient_subset_index_explode(batch, observations)

In [ ]:
print('Current consolidated table dimensions:', batch.shape)

In [13]:
del observations

In [ ]:
batch = batch_merge (batch, df2)

In [ ]:
del df2

In [ ]:
batch = tidy_up (batch, cols2exclude)

In [12]:
batch.dtypes

MARITAL                           uint8
GENDER                            uint8
m_since_birth                    uint16
RACE_asian                        uint8
RACE_black                        uint8
                                  ...  
White oak IgE Ab in Serum_237    uint32
White oak IgE Ab in Serum_238    uint32
White oak IgE Ab in Serum_239    uint32
White oak IgE Ab in Serum_240    uint32
White oak IgE Ab in Serum_241    uint32
Length: 21655, dtype: object

In [ ]:
batch.to_pickle('batch1.pkl')

---
---

## Procedures

In [15]:
procedures = pd.read_pickle('procedure_encounters.pkl')

In [16]:
display(procedures.head())

,Admission to burn unit,Admission to long stay hospital,Admission to neurosurgical department,Admission to orthopedic department,Admission to trauma surgery department,Allergy screening test,Amputation of left arm,Amputation of left foot,Amputation of left hand,Amputation of left leg,...,Surgical manipulation of joint of knee,Surgical manipulation of shoulder joint,Suture open wound,Thoracentesis (procedure),Throat culture (procedure),Total knee replacement,Total replacement of hip,Transplant of lung (procedure),Vasectomy,PATIENT
1516584,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33f33990-ae8b-4be8-938f-e47ad473abfe
1600279,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,36d131ee-dd5b-4acb-acbe-19961c32c099
1516585,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33f33990-ae8b-4be8-938f-e47ad473abfe
1600280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,36d131ee-dd5b-4acb-acbe-19961c32c099
2987976,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,660bec03-9e58-47f2-98b9-2f1c564f3838


In [ ]:
batch = pd.read_pickle('batch1.pkl')

In [17]:
cols2exclude = list(batch.columns)

In [20]:
df2 = patient_subset_index_explode(batch, procedures)

subsetting...
column types: Admission to burn unit                    Sparse[int64, 0]
Admission to long stay hospital           Sparse[int64, 0]
Admission to neurosurgical department     Sparse[int64, 0]
Admission to orthopedic department        Sparse[int64, 0]
Admission to trauma surgery department    Sparse[int64, 0]
                                                ...       
Total knee replacement                    Sparse[int64, 0]
Total replacement of hip                  Sparse[int64, 0]
Transplant of lung (procedure)            Sparse[int64, 0]
Vasectomy                                 Sparse[int64, 0]
PATIENT                                             object
Length: 84, dtype: object
Subset table dimensions: (562787, 84)
Processing column Admission to burn unit...
Processing column Admission to long stay hospital...
Processing column Admission to neurosurgical department...
Processing column Admission to orthopedic department...
Processing column Admission to trauma surgery d

C:\Users\Admin\AppData\Local\Temp\ipykernel_14660\2292251919.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tab[col] = tab[col].sparse.to_dense().astype(np.uint32)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14660\2292251919.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tab['encounter_n'] = tab.groupby('PATIENT').cumcount()+1


Pivoting table and setting "PATIENT" as index
(92939, 9711)
column types: Admission to burn unit_1    float64
Admission to burn unit_2    float64
Admission to burn unit_3    float64
Admission to burn unit_4    float64
Admission to burn unit_5    float64
                             ...   
Vasectomy_113               float64
Vasectomy_114               float64
Vasectomy_115               float64
Vasectomy_116               float64
Vasectomy_117               float64
Length: 9711, dtype: object
Filling in nulls with 0s


In [21]:
del procedures

In [22]:
print('Current consolidated table dimensions:', batch.shape)

Current consolidated table dimensions: (119293, 21655)


In [23]:
batch = batch_merge (batch, df2)

Merging...
Merged!
New dimensions: (119293, 31366)


In [24]:
del df2

In [25]:
batch = tidy_up (batch, cols2exclude)

Filling up with 0s
Processing column Pulmonary rehabilitation (regime/therapy)_1_x...
Processed column Pulmonary rehabilitation (regime/therapy)_1_x.
Processing column Pulmonary rehabilitation (regime/therapy)_2_x...
Processed column Pulmonary rehabilitation (regime/therapy)_2_x.
Processing column Pulmonary rehabilitation (regime/therapy)_3_x...
Processed column Pulmonary rehabilitation (regime/therapy)_3_x.
Processing column Pulmonary rehabilitation (regime/therapy)_4_x...
Processed column Pulmonary rehabilitation (regime/therapy)_4_x.
Processing column Pulmonary rehabilitation (regime/therapy)_5_x...
Processed column Pulmonary rehabilitation (regime/therapy)_5_x.
Processing column Pulmonary rehabilitation (regime/therapy)_6_x...
Processed column Pulmonary rehabilitation (regime/therapy)_6_x.
Processing column Pulmonary rehabilitation (regime/therapy)_7_x...
Processed column Pulmonary rehabilitation (regime/therapy)_7_x.
Processing column Pulmonary rehabilitation (regime/therapy)_8_x.

Processed column Admission to burn unit_36.
Processing column Admission to burn unit_37...
Processed column Admission to burn unit_37.
Processing column Admission to burn unit_38...
Processed column Admission to burn unit_38.
Processing column Admission to burn unit_39...
Processed column Admission to burn unit_39.
Processing column Admission to burn unit_40...
Processed column Admission to burn unit_40.
Processing column Admission to burn unit_41...
Processed column Admission to burn unit_41.
Processing column Admission to burn unit_42...
Processed column Admission to burn unit_42.
Processing column Admission to burn unit_43...
Processed column Admission to burn unit_43.
Processing column Admission to burn unit_44...
Processed column Admission to burn unit_44.
Processing column Admission to burn unit_45...
Processed column Admission to burn unit_45.
Processing column Admission to burn unit_46...
Processed column Admission to burn unit_46.
Processing column Admission to burn unit_47...

Processed column Admission to long stay hospital_8.
Processing column Admission to long stay hospital_9...
Processed column Admission to long stay hospital_9.
Processing column Admission to long stay hospital_10...
Processed column Admission to long stay hospital_10.
Processing column Admission to long stay hospital_11...
Processed column Admission to long stay hospital_11.
Processing column Admission to long stay hospital_12...
Processed column Admission to long stay hospital_12.
Processing column Admission to long stay hospital_13...
Processed column Admission to long stay hospital_13.
Processing column Admission to long stay hospital_14...
Processed column Admission to long stay hospital_14.
Processing column Admission to long stay hospital_15...
Processed column Admission to long stay hospital_15.
Processing column Admission to long stay hospital_16...
Processed column Admission to long stay hospital_16.
Processing column Admission to long stay hospital_17...
Processed column Admis

Processed column Admission to long stay hospital_84.
Processing column Admission to long stay hospital_85...
Processed column Admission to long stay hospital_85.
Processing column Admission to long stay hospital_86...
Processed column Admission to long stay hospital_86.
Processing column Admission to long stay hospital_87...
Processed column Admission to long stay hospital_87.
Processing column Admission to long stay hospital_88...
Processed column Admission to long stay hospital_88.
Processing column Admission to long stay hospital_89...
Processed column Admission to long stay hospital_89.
Processing column Admission to long stay hospital_90...
Processed column Admission to long stay hospital_90.
Processing column Admission to long stay hospital_91...
Processed column Admission to long stay hospital_91.
Processing column Admission to long stay hospital_92...
Processed column Admission to long stay hospital_92.
Processing column Admission to long stay hospital_93...
Processed column Ad

Processed column Admission to neurosurgical department_38.
Processing column Admission to neurosurgical department_39...
Processed column Admission to neurosurgical department_39.
Processing column Admission to neurosurgical department_40...
Processed column Admission to neurosurgical department_40.
Processing column Admission to neurosurgical department_41...
Processed column Admission to neurosurgical department_41.
Processing column Admission to neurosurgical department_42...
Processed column Admission to neurosurgical department_42.
Processing column Admission to neurosurgical department_43...
Processed column Admission to neurosurgical department_43.
Processing column Admission to neurosurgical department_44...
Processed column Admission to neurosurgical department_44.
Processing column Admission to neurosurgical department_45...
Processed column Admission to neurosurgical department_45.
Processing column Admission to neurosurgical department_46...
Processed column Admission to ne

...

Processed column Total knee replacement_41.
Processing column Total knee replacement_42...
Processed column Total knee replacement_42.
Processing column Total knee replacement_43...
Processed column Total knee replacement_43.
Processing column Total knee replacement_44...
Processed column Total knee replacement_44.
Processing column Total knee replacement_45...
Processed column Total knee replacement_45.
Processing column Total knee replacement_46...
Processed column Total knee replacement_46.
Processing column Total knee replacement_47...
Processed column Total knee replacement_47.
Processing column Total knee replacement_48...
Processed column Total knee replacement_48.
Processing column Total knee replacement_49...
Processed column Total knee replacement_49.
Processing column Total knee replacement_50...
Processed column Total knee replacement_50.
Processing column Total knee replacement_51...
Processed column Total knee replacement_51.
Processing column Total knee replacement_52...

Processed column Total replacement of hip_15.
Processing column Total replacement of hip_16...
Processed column Total replacement of hip_16.
Processing column Total replacement of hip_17...
Processed column Total replacement of hip_17.
Processing column Total replacement of hip_18...
Processed column Total replacement of hip_18.
Processing column Total replacement of hip_19...
Processed column Total replacement of hip_19.
Processing column Total replacement of hip_20...
Processed column Total replacement of hip_20.
Processing column Total replacement of hip_21...
Processed column Total replacement of hip_21.
Processing column Total replacement of hip_22...
Processed column Total replacement of hip_22.
Processing column Total replacement of hip_23...
Processed column Total replacement of hip_23.
Processing column Total replacement of hip_24...
Processed column Total replacement of hip_24.
Processing column Total replacement of hip_25...
Processed column Total replacement of hip_25.
Proc

Processed column Total replacement of hip_103.
Processing column Total replacement of hip_104...
Processed column Total replacement of hip_104.
Processing column Total replacement of hip_105...
Processed column Total replacement of hip_105.
Processing column Total replacement of hip_106...
Processed column Total replacement of hip_106.
Processing column Total replacement of hip_107...
Processed column Total replacement of hip_107.
Processing column Total replacement of hip_108...
Processed column Total replacement of hip_108.
Processing column Total replacement of hip_109...
Processed column Total replacement of hip_109.
Processing column Total replacement of hip_110...
Processed column Total replacement of hip_110.
Processing column Total replacement of hip_111...
Processed column Total replacement of hip_111.
Processing column Total replacement of hip_112...
Processed column Total replacement of hip_112.
Processing column Total replacement of hip_113...
Processed column Total replace

Processed column Transplant of lung (procedure)_66.
Processing column Transplant of lung (procedure)_67...
Processed column Transplant of lung (procedure)_67.
Processing column Transplant of lung (procedure)_68...
Processed column Transplant of lung (procedure)_68.
Processing column Transplant of lung (procedure)_69...
Processed column Transplant of lung (procedure)_69.
Processing column Transplant of lung (procedure)_70...
Processed column Transplant of lung (procedure)_70.
Processing column Transplant of lung (procedure)_71...
Processed column Transplant of lung (procedure)_71.
Processing column Transplant of lung (procedure)_72...
Processed column Transplant of lung (procedure)_72.
Processing column Transplant of lung (procedure)_73...
Processed column Transplant of lung (procedure)_73.
Processing column Transplant of lung (procedure)_74...
Processed column Transplant of lung (procedure)_74.
Processing column Transplant of lung (procedure)_75...
Processed column Transplant of lung (

Processed column Vasectomy_51.
Processing column Vasectomy_52...
Processed column Vasectomy_52.
Processing column Vasectomy_53...
Processed column Vasectomy_53.
Processing column Vasectomy_54...
Processed column Vasectomy_54.
Processing column Vasectomy_55...
Processed column Vasectomy_55.
Processing column Vasectomy_56...
Processed column Vasectomy_56.
Processing column Vasectomy_57...
Processed column Vasectomy_57.
Processing column Vasectomy_58...
Processed column Vasectomy_58.
Processing column Vasectomy_59...
Processed column Vasectomy_59.
Processing column Vasectomy_60...
Processed column Vasectomy_60.
Processing column Vasectomy_61...
Processed column Vasectomy_61.
Processing column Vasectomy_62...
Processed column Vasectomy_62.
Processing column Vasectomy_63...
Processed column Vasectomy_63.
Processing column Vasectomy_64...
Processed column Vasectomy_64.
Processing column Vasectomy_65...
Processed column Vasectomy_65.
Processing column Vasectomy_66...
Processed column Vasectom

In [26]:
print('Shape for the batch:', batch.shape)
batch.dtypes

Shape for the batch: (119293, 31366)


MARITAL           uint8
GENDER            uint8
m_since_birth    uint16
RACE_asian        uint8
RACE_black        uint8
                  ...  
Vasectomy_113    uint32
Vasectomy_114    uint32
Vasectomy_115    uint32
Vasectomy_116    uint32
Vasectomy_117    uint32
Length: 31366, dtype: object

In [27]:
batch.to_pickle('batch1.pkl')

---
---

## Remove duplicated

In [ ]:
batch.drop_duplicates(inplace=True)
print(batch.shape)

We will also check for presence of any null values.

In [ ]:
print('Total number of nulls after consolidation:', batch.isna().sum().sum())

In [ ]:
batch.to_pickle('batch1.pkl')